In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
import scipy.io as sio
from time import time
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

# device = torch.device("mps") # for apple silicon, cpu is faster
device = torch.device("cpu")
# device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

In [2]:
SAMPLE_DS_PATH = "mg_data/ITER_like_equilibrium_dataset_sample.mat" # sample dataset
FULL_DS_PATH = 'mg_data/ITER_like_equilibrium_dataset.mat' # full dataset
MODEL_SAVE_PATH = "mg_data/mg_planet.pth"
EPOCHS = 15
BATCH_SIZE = 16
LEARNING_RATE = 1e-4

In [3]:
class PlaNetDataset(Dataset):
    def __init__(self, ds_path):
        ds = sio.loadmat(ds_path)
        self.rr_pix = ds["RR_pixels"] # radial position of pixels (64, 64)
        self.zz_pix = ds["ZZ_pixels"] # vertical position of pixels (64, 64)
        self.input_currs = ds["DB_coils_curr_test_ConvNet"] # input currents (n,14)
        self.psi = ds["DB_psi_pixel_test_ConvNet"] # magnetic flux (n, 64, 64)
        # self.psi = self.psi.transpose(0, 2, 1).reshape(-1, 64*64) # transpose and flatten
        self.psi = self.psi.transpose(0, 2, 1) # transpose (matlab is column major)
        self.input_currs = torch.tensor(self.input_currs, dtype=torch.float32) # convert to tensor
        self.psi = torch.tensor(self.psi, dtype=torch.float32) # convert to tensor
    def __len__(self):
        return len(self.psi)
    def __getitem__(self, idx):
        return self.input_currs[idx], self.psi[idx]

In [4]:
# class PlaNet(torch.nn.Module): # simple fully connected neural network -> weak > loss:33
#     def __init__(self):
#         super(PlaNet, self).__init__()
#         self.n = 8
#         self.fc1 = torch.nn.Linear(14, self.n)
#         self.fc2 = torch.nn.Linear(self.n, self.n)
#         self.fc3 = torch.nn.Linear(self.n, 64*64)
        
#     def forward(self, x):
#         x = torch.relu(self.fc1(x))
#         x = torch.relu(self.fc2(x))
#         x = self.fc3(x)
#         x = x.view(-1, 64, 64)
#         return x

In [5]:
class PlaNet(torch.nn.Module): # transpose convolutional neural network -> stronger, fast, but artifacts > loss:0.49
    def __init__(self):
        super(PlaNet, self).__init__()
        self.fc1 = torch.nn.Linear(14, 32)
        self.fc2 = torch.nn.Linear(32, 64)
        self.unconv1 = torch.nn.ConvTranspose2d(64, 32, kernel_size=3)
        self.unconv2 = torch.nn.ConvTranspose2d(32, 16, kernel_size=3, stride=2)
        self.unconv3 = torch.nn.ConvTranspose2d(16, 8, kernel_size=3, stride=2)
        self.unconv4 = torch.nn.ConvTranspose2d(8, 4, kernel_size=3, stride=2)
        self.unconv5 = torch.nn.ConvTranspose2d(4, 1, kernel_size=4, stride=2)
    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = x.view(-1, 64, 1, 1)
        x = torch.relu(self.unconv1(x))
        x = torch.relu(self.unconv2(x))
        x = torch.relu(self.unconv3(x))
        x = torch.relu(self.unconv4(x))
        x = self.unconv5(x)
        x = x.view(-1, 64, 64)
        return x

In [6]:
# class PlaNet(torch.nn.Module): # upsample convolutional neural network -> slower -> but smoother
#     def __init__(self):
#         super(PlaNet, self).__init__()
#         self.interp = 'bilinear' # 'nearest' or 'bilinear'
#         self.fc1 = torch.nn.Linear(14, 32)
#         self.fc2 = torch.nn.Linear(32, 64)
#         self.unconv1 = torch.nn.Sequential(
#             torch.nn.Upsample(scale_factor=4, mode=self.interp),
#             torch.nn.Conv2d(64, 32, kernel_size=3, padding=1),
#             torch.nn.ReLU(),
#             torch.nn.Upsample(scale_factor=2, mode=self.interp),
#             torch.nn.Conv2d(32, 16, kernel_size=3, padding=1),
#             torch.nn.ReLU(),
#             torch.nn.Upsample(scale_factor=2, mode=self.interp),
#             torch.nn.Conv2d(16, 8, kernel_size=3, padding=1),
#             torch.nn.ReLU(),
#             torch.nn.Upsample(scale_factor=2, mode=self.interp),
#             torch.nn.Conv2d(8, 4, kernel_size=3, padding=1),
#             torch.nn.ReLU(),
#             torch.nn.Upsample(scale_factor=2, mode=self.interp),
#             torch.nn.Conv2d(4, 1, kernel_size=3, padding=1),
#         )
#     def forward(self, x):
#         x = torch.relu(self.fc1(x))
#         x = torch.relu(self.fc2(x))
#         x = x.view(-1, 64, 1, 1)
#         x = self.unconv1(x)
#         x = x.view(-1, 64, 64)
#         return x

In [7]:
# class PlaNet(torch.nn.Module): # mix of the previous 2, similar results, slow
#     def __init__(self):
#         super(PlaNet, self).__init__()
#         self.fc1 = torch.nn.Linear(14, 32)
#         self.fc2 = torch.nn.Linear(32, 64)
#         self.unconv1 = torch.nn.Sequential(
#             torch.nn.ConvTranspose2d(64, 32, kernel_size=3),
#             torch.nn.ReLU(),
#             torch.nn.ConvTranspose2d(32, 16, kernel_size=3, stride=2),
#             torch.nn.ReLU(),
#             torch.nn.ConvTranspose2d(16, 8, kernel_size=3, stride=2),
#             torch.nn.ReLU(),
#             torch.nn.ConvTranspose2d(8, 4, kernel_size=3, stride=2),
#             torch.nn.ReLU(),
#             torch.nn.ConvTranspose2d(4, 2, kernel_size=4, stride=2), # 4,2
#             torch.nn.ReLU(),
#             torch.nn.ConvTranspose2d(2, 1, kernel_size=3, stride=2),
#             torch.nn.ReLU(),
#             torch.nn.Conv2d(1, 1, kernel_size=3, stride=2)
#         )
#     def forward(self, x):
#         x = torch.relu(self.fc1(x))
#         x = torch.relu(self.fc2(x))
#         x = x.view(-1, 64, 1, 1)
#         x = self.unconv1(x)
#         x = x.view(-1, 64, 64)
#         return x

In [ ]:
train_ds, val_ds = PlaNetDataset(FULL_DS_PATH), PlaNetDataset(SAMPLE_DS_PATH)
train_dl = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True) # initialize DataLoader
val_dl = DataLoader(val_ds, batch_size=BATCH_SIZE, shuffle=False)  
model = PlaNet()  # instantiate model
model.to(device) # move model to device
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
loss_fn = torch.nn.MSELoss() # Mean Squared Error Loss
best_loss = float('inf') # initialize best loss
for epoch in range(EPOCHS): 
    epoch_time = time()
    model.train()
    trainloss, evalloss = [], []
    for input_currs, psi in tqdm(train_dl, desc=f"Epoch {epoch+1}/{EPOCHS}", leave=False):
    # for input_currs, psi in train_dl:
        input_currs, psi = input_currs.to(device), psi.to(device) # move to device
        optimizer.zero_grad()
        psi_pred = model(input_currs)
        loss = loss_fn(psi_pred, psi)
        loss.backward()
        optimizer.step()
        trainloss.append(loss.item())
    model.eval()
    with torch.no_grad():
        for input_currs, psi in val_dl:
            input_currs, psi = input_currs.to(device), psi.to(device) # move to device
            psi_pred = model(input_currs)
            loss = loss_fn(psi_pred, psi)
            evalloss.append(loss.item())
    print(f"Ep {epoch+1}: Train Loss: {sum(trainloss)/len(trainloss):.4f}, Eval Loss: {sum(evalloss)/len(evalloss):.4f}, Time: {time()-epoch_time:.2f}s,", end=" ")
    if sum(evalloss)/len(evalloss) < best_loss:
        best_loss = sum(evalloss)/len(evalloss)
        torch.save(model.state_dict(), MODEL_SAVE_PATH)
        print("saved")

In [ ]:
model = PlaNet()
model.load_state_dict(torch.load(MODEL_SAVE_PATH))
model.eval()
ds = PlaNetDataset(SAMPLE_DS_PATH)
for i in np.random.randint(0, len(ds), 10):  
    input_currs, psi = ds[i]
    psi_pred = model(input_currs.unsqueeze(0))
    psi_pred = psi_pred.detach().numpy().reshape(64, 64)
    psi = psi.detach().numpy().reshape(64, 64)
    fig, axs = plt.subplots(1, 4, figsize=(15, 5))
    ext = [ds.rr_pix.min(), ds.rr_pix.max(), ds.zz_pix.min(), ds.zz_pix.max()]
    rr, zz = ds.rr_pix, ds.zz_pix  # radial and vertical positions of pixels
    bmin, bmax = np.min([psi, psi_pred]), np.max([psi, psi_pred])
    err = np.abs(psi - psi_pred)*100/bmax

    im0 = axs[0].imshow(psi, extent=ext, vmin=bmin, vmax=bmax)
    axs[0].set_title("Actual")
    axs[0].set_aspect('equal')
    fig.colorbar(im0, ax=axs[0]) 

    im1 = axs[1].imshow(psi_pred, extent=ext, vmin=bmin, vmax=bmax)
    axs[1].set_title("Predicted")
    axs[1].set_aspect('equal')
    fig.colorbar(im1, ax=axs[1])

    im2 = axs[2].imshow(err, extent=ext)
    axs[2].set_title("Error %")
    axs[2].set_aspect('equal')
    fig.colorbar(im2, ax=axs[2])

    c0 = axs[3].contour(rr, zz, psi, levels=15, cmap='viridis', linestyles='dashed')
    c1 = axs[3].contour(rr, zz, psi_pred, levels=10, cmap='viridis')
    axs[3].set_title("Contours")
    axs[3].set_aspect('equal')

    plt.show()